- These codes splits the digital archived copies into page units. 

- The codes are executed in the following order.
    1. Remove redundant spaces around a digital copies.
    
        <=Removes the areas of the image covered with while color.
        
    2. Cuts the page at the border of the two pages.
        
        Borders are detected by searching for a black ertical line.

Code 1: Define the functions and install packages

In [6]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import cv2
import os


def Split(Page):    
    img = cv2.imread(path+"Page"+Page+".jpg")
    hh, ww = img.shape[:2]
    # convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # threshold gray image
    thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

    # count number of non-zero pixels in each row. 
    countCol = np.count_nonzero(thresh, axis=0)
    countRow = np.count_nonzero(thresh, axis=1)

    ### Find location of biggest jumps/falls.
    ###Top and Bottom
    peaks, _ = find_peaks(countRow, distance=15)
    HeightTop=abs(np.diff(countRow[peaks][peaks<hh//5]))
    HeightBtm=abs(np.diff(countRow[peaks][peaks>4*hh//5]))
    max1Height=np.sort(HeightTop)[-1]
    max2Height=np.sort(HeightBtm)[-1]
    X=peaks[ : -1][abs(np.diff(countRow[peaks]))==max1Height][0]
    Y=peaks[ : -1][abs(np.diff(countRow[peaks]))==max2Height][0]
    Top=min([X,Y])
    Btm=max([X,Y])

    ###Left and Right
    peaks, _ = find_peaks(countCol, distance=15)
    HeightLeft=abs(np.diff(countCol[peaks][peaks<ww//5]))
    HeightRight=abs(np.diff(countCol[peaks][peaks>4*ww//5]))
    max1Height=np.sort(HeightLeft)[-1]
    max2Height=np.sort(HeightRight)[-1]
    V=peaks[ : -1][abs(np.diff(countCol[peaks]))==max1Height][0]
    Z=peaks[ : -1][abs(np.diff(countCol[peaks]))==max2Height][0]
    Left=min([V,Z])
    Right=max([V,Z])

    cropped=img[Top:Btm, Left:Right] 
    #cv2.imshow("Result", cropped)
    #cv2.waitKey(0)
    
    #Code for cutting image Side-by-Side
    # read image+crop image
    crop_img=img[Top:Btm, Left:Right]
    HH, WW = crop_img.shape[:2]
    # convert to grayscale
    gray = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
    # threshold gray image
    thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

    # count number of non-zero pixels in each column and row. 
    countCol = np.count_nonzero(thresh, axis=0)
    countRow = np.count_nonzero(thresh, axis=1)

    ### This finds the height of the lowest valley
    peaks, _ = find_peaks(-countCol, distance=15)
    maxHeight=np.sort(abs(np.diff(countCol[peaks][(peaks<4*WW//7) & (peaks>3*WW//7)])))[-1]
    Border=peaks[ : -1][abs(np.diff(countCol[peaks]))==maxHeight][0]

    ### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
    result = crop_img.copy()
    LeftImage=result[0:HH,0:Border]
    RighImage=result[0:HH,Border:WW]
    #cv2.imshow("LeftImage", LeftImage)
    #cv2.imshow("RightImage", RighImage)
    #cv2.waitKey(0)

    return(list((LeftImage,RighImage)))

Code 2: Define Years

In [4]:
Year=1934

Code 3: We need to create a seperate directory for each page of the archived copy.
Check the starting and ending page of the staff directory and define a variable with the numbers.

In [7]:
StrPage=46
EndPage=116

#Set directory path
save_path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+str(Year)+"\\"

#Create directory for each page
for Num in range(StrPage,EndPage+1):
    Page="Page"+"{:03d}".format(2*(Num-46))
    path2 = os.path.join(save_path, Page)
    os.mkdir(path2)
    
    Page="Page"+"{:03d}".format(2*(Num-46)+1)
    path2 = os.path.join(save_path, Page)
    os.mkdir(path2)

Code 4: Splitting copies into pages.
Each page is saved into the corresponding directory that we made on Code 2.

In [8]:
# read image+crop image
import os
path=r'C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs/Raw_Data\\Metropolitan_DA\\1934\\Line\\'
for Num in range(StrPage,EndPage):
    try:
        print(Num)
        Righ=Split("{:03d}".format(Num))[1]
        Left=Split("{:03d}".format(Num))[0]

        Page="Page"+"{:03d}".format(2*(Num-46))
        path2 = os.path.join(save_path, Page)
        print(path2+"\\"+Page+".jpg")
        cv2.imwrite(path2+"\\"+Page+".jpg", Righ)

        Page="Page"+"{:03d}".format(2*(Num-46)+1)
        path2 = os.path.join(save_path, Page)
        print(path2+"\\"+Page+".jpg")
        cv2.imwrite(path2+"\\"+Page+".jpg", Left)
    except IndexError:
        continue
    

46
C:\Users\Keitaro Ninomiya\Box\Research Notes (keitaro2@illinois.edu)\Tokyo_Jobs\Raw_Data\Splited\1934\Page000\Page000.jpg
C:\Users\Keitaro Ninomiya\Box\Research Notes (keitaro2@illinois.edu)\Tokyo_Jobs\Raw_Data\Splited\1934\Page001\Page001.jpg
47
C:\Users\Keitaro Ninomiya\Box\Research Notes (keitaro2@illinois.edu)\Tokyo_Jobs\Raw_Data\Splited\1934\Page002\Page002.jpg
C:\Users\Keitaro Ninomiya\Box\Research Notes (keitaro2@illinois.edu)\Tokyo_Jobs\Raw_Data\Splited\1934\Page003\Page003.jpg
48
C:\Users\Keitaro Ninomiya\Box\Research Notes (keitaro2@illinois.edu)\Tokyo_Jobs\Raw_Data\Splited\1934\Page004\Page004.jpg
C:\Users\Keitaro Ninomiya\Box\Research Notes (keitaro2@illinois.edu)\Tokyo_Jobs\Raw_Data\Splited\1934\Page005\Page005.jpg
49
C:\Users\Keitaro Ninomiya\Box\Research Notes (keitaro2@illinois.edu)\Tokyo_Jobs\Raw_Data\Splited\1934\Page006\Page006.jpg
C:\Users\Keitaro Ninomiya\Box\Research Notes (keitaro2@illinois.edu)\Tokyo_Jobs\Raw_Data\Splited\1934\Page007\Page007.jpg
50
C:\Users\